In [1]:
import math as m
import h5py 
import numpy as np
import matplotlib.pyplot as plt

In [106]:
import numpy as np
import h5py
class Antenna:
    """ antenna with location and timeseries of Efield"""
    def __init__(self,x,y,z,name=""):
        self.x = x
        self.y = y
        self.z = z
        self.name = name

class REvent:
    """ An interface class for reading simulations """
    def __init__(self,filename):
        self.reasfile = filename
        if '.reas' in filename:
            self.reasfile = filename
            self.parse_reas()
        elif '.hdf5' in filename:
            self.hdf5file = filename
            self.parse_hdf5()

    def parse_reas(self):
        """ parse *reas file """
        f = open(self.reasfile)
        lines = f.readlines()
        for l in lines:
            if 'ShowerZenithAngle' in l:
                self.zenith = float(l.split()[2])
            if 'ShowerAzimuthAngle' in l:
                self.azimuth = float(l.split()[2]) + 180
                if self.azimuth > 360:
                    self.azimuth = self.azimuth - 360
            if 'DepthOfShowerMaximum' in l:
                self.xmax = float(l.split()[2])
            if 'PrimaryParticleEnergy' in l:
                self.energy = float(l.split()[2])/1e18
            if 'PrimaryParticleType' in l:
                self.part_id = int(l.split()[2])
            if 'MagneticFieldInclinationAngle' in l:
                self.Binc = float(l.split()[2])
            if 'MagneticFieldStrength' in l:
                self.Bmag = float(l.split()[2])
        self.parse_reas_antenas()
        self.set_unit_vectors()

    def parse_reas_antenas(self):
        """Parse the antennas by reas """
        self.listfile = self.reasfile.replace('.reas','.list')
        f = open(self.listfile)
        lines = f.readlines()
        self.antennas = []
        print('reading ' +  str(len(lines)) + ' antennas')
        for l in lines:
            a = Antenna(float(l.split()[2])/100,float(l.split()[3])/100,float(l.split()[4])/100,l.split()[5])
            efile = self.listfile.replace('.list','_coreas/raw_'+ a.name + '.dat')
            a.t, a.Ex, a.Ey, a.Ez = np.loadtxt(efile,unpack=True)
            a.t = np.asarray(a.t)*1e9 #use nanoseconds
            a.Ex = np.asarray(a.Ex) * 2.99792458e10 #cgs statvolt/cm volt -> mu V/m (SI)
            a.Ey = np.asarray(a.Ey) * 2.99792458e10 #cgs statvolt/cm volt-> mu V/m (SI)
            a.Ez = np.asarray(a.Ez) * 2.99792458e10 #cgs statvolt/cm volt-> mu V/m (SI)
            self.antennas.append(a)

    def parse_hdf5(self):
        f = h5py.File(self.hdf5file, "r")
        self.antennas = []
        ai = f[list(f.keys())[-1] + '/AntennaInfo']
        for ai_ in ai:
            a = Antenna(ai_[1],ai_[2],ai_[3],ai_[0].decode('UTF-8'))
            self.antennas.append(a)
        print('reading ' +  str(len(self.antennas)) + ' antennas')

        traces = f[list(f.keys())[-1] + '/AntennaTraces']
        for tr in traces:
            for a in self.antennas:
                if a.name == tr:
                    break
            a.t = []
            a.Ex = []
            a.Ey = []
            a.Ez =[]
            for tup in traces[tr + '/efield']:
                a.t.append(tup[0])
                a.Ex.append(tup[1])
                a.Ey.append(tup[2])
                a.Ez.append(tup[3])
            a.t = np.asarray(a.t)
            a.Ex = np.asarray(a.Ex)
            a.Ey = np.asarray(a.Ey)
            a.Ez = np.asarray(a.Ez)

        ei = f[list(f.keys())[-1] + '/EventInfo']
        ei=ei[0]
        self.zenith = 180.-ei[4]
        self.azimuth = ei[5]+180
        if self.azimuth >= 360:
            self.azimuth -= 360
        self.xmax = ei[9]
        self.energy = ei[3]
        self.part_id = ei[2].decode('UTF-8')
        self.Bmag = ei[16]
        self.Binc = ei[17]
        self.Bdec = ei[18]
        self.ground=ei[11]
        self.dist_xmax=ei[6]
        self.set_unit_vectors()

In [6]:
f = h5py.File('Stshp_Proton_3.98_38.2_90.0_22.hdf5', "r")
antennas = []
ai = f[list(f.keys())[-1] + '/AntennaInfo']
for ai_ in ai:
    a = Antenna(ai_[1],ai_[2],ai_[3],ai_[0].decode('UTF-8'))
    antennas.append(a)
print('read ' +  str(len(antennas)) + 'antennas')
traces = f[list(f.keys())[-1] + '/AntennaTraces']
for tr in traces:
    for a in antennas:
        if a.name == tr:
            break
    a.t = []
    a.Ex = []
    a.Ey = []
    a.Ez =[]
    for tup in traces[tr + '/efield']:
        a.t.append(tup[0])
        a.Ex.append(tup[1])
        a.Ey.append(tup[2])
        a.Ez.append(tup[3])

ei = f[list(f.keys())[-1] + '/EventInfo']
ei=ei[0]
zenith = 180.-ei[4]
azimuth = ei[5]
xmax = ei[9]
energy = ei[3]
part_id = ei[2].decode('UTF-8')
Bmag = ei[16]
Binc = ei[17]
Bdec = ei[18]
ground=ei[11]
dist_xmax=ei[6]

read 176antennas


In [107]:
Binc

60.79

In [116]:
def vxvxb(zenith,azimuth,Binc):
    zen = np.deg2rad(zenith)
    azi = np.deg2rad(azimuth)
    Binc= np.deg2rad(Binc)
    v = [-np.sin(zenith)*np.cos(azimuth),-np.sin(zenith)*np.sin(azimuth),-np.cos(zenith)]
    # in the direction opp to the shower axis
    B = [np.cos(Binc), 0.0 ,-np.sin(Binc)]
    vxB=np.cross(v,B)
    vxvxB=np.cross(v,vxB)
    print(v)
    print(B)
    print(vxB)
    print(np.cross(vxB,vxvxB))
    print(np.dot(vxvxB,vxB))
    print(np.dot(v,vxB))
    
       
vxvxb(zenith,azimuth,Binc)

[-0.4983920399869236, 0.08913196583586219, -0.8623577373363721]
[0.4880120053510488, 0.0, -0.8728369164014821]
[-0.07779767 -0.8558559  -0.04349747]
[-0.36902633  0.06599632 -0.63851884]
0.0
0.0
